In [4]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
#Task 4
Z=2
N = 1000
a = float(0)
b = float(5) #This can be referred to as rmax in some cases
h = (b-a)/N
rho = np.arange(1,N+1)*h

initial_guess =1/np.pi*Z**3*np.exp(-2*Z*rho) 

def generate_wavefunction(rho,V,Z):
    #Returns a wave function and epsilon energy
    diagonal_term = h**-2 - Z*rho**-1 + V
    A = np.diag(diagonal_term,0)+np.diag(-np.ones(N-1)/(2*h**2),1)+np.diag(-np.ones(N-1)/(2*h**2),-1)
    #Eigenvalues & eigenvectors:
    (Eigenvalues,Eigenvectors) = np.linalg.eig(A)
  
    # X contains N eigenvalues, build the wavefunction
    #with the lowest eigenvalue:
    
    minimized_energy_vector = Eigenvectors[:,np.argmin(Eigenvalues)]
    temp = minimized_energy_vector
    temp = temp/(np.sqrt(np.trapz(temp**2,rho))*np.sign(temp[0]))
    
    wavefunction = 1/np.sqrt(4*np.pi)*temp*rho**-1
    
    #Normalize
    #wavefunction = wavefunction/(np.trapz(wavefunction,rho))
    epsilon = np.min(Eigenvalues).real
    
    return (epsilon,wavefunction)


def generate_energy(epsilon, V_H, density, Z):
    E = Z*epsilon-4*Z*\
        np.pi*np.trapz((V_H*density/2)*rho**2,rho)  
    return E

def generate_hartree(rho,N,density):
    A = np.diag(-2*np.ones(N),0)+np.diag(np.ones(N-1),1) + np.diag(np.ones(N-1),-1)
    potential = -4*np.pi*h**2*rho*density
    potential[-1] -= 1.5
    
    V_H = (np.linalg.solve(A,potential))*rho**-1
    return V_H

def self_consistency_solver(iterations,threshhold):
    E = np.zeros(iterations)
    density = initial_guess
    V_H = Z*generate_hartree(rho,N,density)
    for i in range(iterations):
        #print(f'Iteration: {i}',end='\r')
        
        psi = generate_wavefunction(rho,V_H,Z)[1]
        epsilon = generate_wavefunction(rho,V_H,Z)[0]
        
        density = np.abs(psi)**2
        V_H = Z*generate_hartree(rho,N,density)
        E[i] = generate_energy(epsilon, V_H,density,Z)

        print(f'Iteration: {i}, Diff%: {(E[i]-E[i-1])/threshhold}',end='\r')

        if np.abs(E[i]-E[i-1]) < threshhold:
            print("E_g = " + str( E[i]))
            it_fin = i # Iterations to reach convergence
            break
        
    print(f'Total amount of iterations: {it_fin}')


iterations = 500
threshhold = 1e-5
self_consistency_solver(iterations,threshhold)


E_g = -1.7447854493528798.9708254261386883
Total amount of iterations: 263
